In [7]:
#load required library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.callbacks import TensorBoard
import time

In [2]:
#label data

#load data from this link  - https://www.microsoft.com/en-us/download/details.aspx?id=54765

DataDir = "./cat_dog/PetImages"

Categories = ["Dog","Cat"]

data = []

def create_data():
    for category in Categories:
        class_num = Categories.index(category)
        path = os.path.join(DataDir,category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_arr = cv2.resize(img_arr,(50,50))
                data.append([new_arr,class_num])
            except Exception as e:
                pass
            

create_data()

In [5]:
#give name for tensorboard graphs..

Name = "Cats-vs-Dogs-{}".format(int(time.time()))

In [8]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(Name))

In [9]:
import random
random.shuffle(data)

In [10]:
X = []
y = []

In [11]:
for features,label in data:
    X.append(features)
    y.append(label)

In [12]:
X = np.array(X).reshape(-1,50,50,1)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPool2D

In [14]:
#scale X with 0 to 1 range
X = X/255.0

In [15]:
#add conv-->maxpool-->conv-->maxpool-->dense-->dense layer
#use optimizer ex: adam, rmsprops
#use activation ex: tanh, relu and final layer sigmoid(binary output)


model = Sequential()

model.add(Conv2D(64, (3,3),input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,batch_size=32,epochs=2,validation_split=0.3,callbacks=[tensorboard])

Train on 17460 samples, validate on 7484 samples
Epoch 1/2
17460/17460 [==============================] - 72s 4ms/step - loss: 0.6331 - acc: 0.6342 - val_loss: 0.5707 - val_acc: 0.6982
Epoch 2/2
17460/17460 [==============================] - 72s 4ms/step - loss: 0.5492 - acc: 0.7266 - val_loss: 0.5164 - val_acc: 0.7480


In [24]:
#load test image to check validation, it must be different sample then training model..

test_image = "./cat_dog/test_images/cat.jpg"
img_size = 50
test_img_arr = cv2.imread(test_image,cv2.IMREAD_GRAYSCALE)
test_new_img = cv2.resize(test_img_arr,(img_size,img_size))
final_img = test_new_img.reshape(-1,img_size,img_size,1)

In [25]:
#predict on test image and print output
Categories[int(model.predict([final_img])[0][0])]

'Cat'

In [28]:
model.save('image_classifier_cnn.model')

In [29]:
#tf.keras.models.load_model('image_classifier_cnn.model')